# 01_bronze_ingestion — Camada Bronze (Raw → Delta)

## Objetivo deste notebook
Este notebook realiza a **ingestão do dataset raw**, em csv, para a camada **Bronze**, da arquitetura Medallion.  
A intenção aqui é preservar os dados o mais próximo possível do formato original, com transformações mínimas, garantindo:
- rastreabilidade da origem;
- possibilidade de reprocessamento para Silver/Gold;
- persistência em formato **Delta** para consultas e evoluções posteriores.

## Fonte dos dados
- Arquivo CSV armazenado no Volume Raw do Unity Catalog:
  - `/Volumes/workspace/default/mvp_chess_raw/club_games_dataset_chess_dot_com.csv`
- O dataset contém registros de partidas de xadrez (Chess.com), incluindo metadados.

## Saídas desta etapa
- Tabela Delta Bronze criada/atualizada:
  - `workspace.default.bronze_games`

## Evidências
Este notebook mostra:
- contagem total de registros e colunas;
- schema;
- amostra dos dados (primeiras linhas);
- validação via SQL (COUNT e preview).


In [0]:
file_info = dbutils.fs.ls("/Volumes/workspace/default/mvp_chess_raw")
csv_path = file_info[0].path

df_raw = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("multiLine", "true")
    .option("quote", '"')
    .option("escape", '"')
    .option("mode", "PERMISSIVE")
    .load(csv_path)
)

print(f"Complete File Info: {file_info}")
print(f"CSV Path: {csv_path}")

print()
print(f"Linhas: {df_raw.count()}")
print(f"Colunas: {len(df_raw.columns)}")

print()
df_raw.printSchema()
display(df_raw.limit(10))


Complete File Info: [FileInfo(path='dbfs:/Volumes/workspace/default/mvp_chess_raw/club_games_dataset_chess_dot_com.csv', name='club_games_dataset_chess_dot_com.csv', size=166670659, modificationTime=1765579542000)]
CSV Path: dbfs:/Volumes/workspace/default/mvp_chess_raw/club_games_dataset_chess_dot_com.csv

Linhas: 66879
Colunas: 14

root
 |-- white_username: string (nullable = true)
 |-- black_username: string (nullable = true)
 |-- white_id: string (nullable = true)
 |-- black_id: string (nullable = true)
 |-- white_rating: integer (nullable = true)
 |-- black_rating: integer (nullable = true)
 |-- white_result: string (nullable = true)
 |-- black_result: string (nullable = true)
 |-- time_class: string (nullable = true)
 |-- time_control: string (nullable = true)
 |-- rules: string (nullable = true)
 |-- rated: boolean (nullable = true)
 |-- fen: string (nullable = true)
 |-- pgn: string (nullable = true)



white_username,black_username,white_id,black_id,white_rating,black_rating,white_result,black_result,time_class,time_control,rules,rated,fen,pgn
-Amos-,miniman2804,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/miniman2804,1708,1608,win,checkmated,daily,1/259200,chess,true,r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R b K f3 1 22,"[Event ""Enjoyable games 2 - Round 1""] [Site ""Chess.com""] [Date ""2013.01.30""] [Round ""-""] [White ""-Amos-""] [Black ""miniman2804""] [Result ""1-0""] [Tournament ""https://www.chess.com/tournament/enjoyable-games-2""] [CurrentPosition ""r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R b K f3 1 22""] [Timezone ""UTC""] [ECO ""E22""] [ECOUrl ""https://www.chess.com/openings/Nimzo-Indian-Defense-Spielmann-Variation""] [UTCDate ""2013.01.30""] [UTCTime ""16:35:14""] [WhiteElo ""1708""] [BlackElo ""1608""] [TimeControl ""1/259200""] [Termination ""-Amos- won by checkmate""] [StartTime ""16:35:14""] [EndDate ""2013.02.01""] [EndTime ""18:14:48""] [Link ""https://www.chess.com/game/daily/64629816""] 1. d4 Nf6 2. c4 e6 3. Nc3 Bb4 4. Qb3 Bxc3+ 5. Qxc3 O-O 6. Bg5 c5 7. dxc5 Nc6 8. Nf3 Qa5 9. Bxf6 gxf6 10. Qxa5 Nxa5 11. e3 Rd8 12. Rd1 Kg7 13. Be2 b6 14. Rd4 bxc5 15. Rg4+ Kh6 16. Bd3 f5 17. Rh4+ Kg6 18. g4 Ba6 19. gxf5+ exf5 20. Ne5+ Kf6 21. Rh6+ Kxe5 22. f4# 1-0"
-Amos-,koltcho69,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/koltcho69,1726,1577,win,resigned,daily,1/172800,chess,true,8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33,"[Event ""Rapid Rats - Board 5""] [Site ""Chess.com""] [Date ""2013.01.19""] [Round ""-""] [White ""-Amos-""] [Black ""koltcho69""] [Result ""1-0""] [Match ""https://www.chess.com/club/matches/219602""] [CurrentPosition ""8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33""] [Timezone ""UTC""] [ECO ""C53""] [ECOUrl ""https://www.chess.com/openings/Giuoco-Piano-Game-Main-Line""] [UTCDate ""2013.01.19""] [UTCTime ""14:29:25""] [WhiteElo ""1726""] [BlackElo ""1577""] [TimeControl ""1/172800""] [Termination ""-Amos- won by resignation""] [StartTime ""14:29:25""] [EndDate ""2013.02.01""] [EndTime ""18:22:03""] [Link ""https://www.chess.com/game/daily/64070770""] 1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5 4. c3 a6 5. d4 exd4 6. cxd4 Be7 7. Qb3 Na5 8. Qc2 Nxc4 9. Qxc4 d6 10. Nc3 c6 11. O-O h6 12. Re1 Nf6 13. d5 c5 14. e5 dxe5 15. Nxe5 O-O 16. Ng6 Re8 17. Rxe7 Rxe7 18. Nxe7+ Qxe7 19. Bf4 b5 20. d6 Qd7 21. Qxc5 Bb7 22. Qc7 Qxc7 23. dxc7 Nd5 24. Rd1 Nxf4 25. Rd8+ Kh7 26. Rxa8 Bxa8 27. c8=Q Bxg2 28. Qxa6 Bf3 29. Qxb5 Nh3+ 30. Kf1 g6 31. Qd7 Ng5 32. h4 Ne6 33. Qxf7+ 1-0"
-Amos-,enhmandah,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/enhmandah,1727,842,win,resigned,daily,1/172800,chess,true,rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/R1B1K2R b KQ - 5 15,"[Event ""CHESS BOARD CLASH - Round 1""] [Site ""Chess.com""] [Date ""2013.02.01""] [Round ""-""] [White ""-Amos-""] [Black ""enhmandah""] [Result ""1-0""] [Tournament ""https://www.chess.com/tournament/just-another-clash""] [CurrentPosition ""rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/R1B1K2R b KQ - 5 15""] [Timezone ""UTC""] [ECO ""D00""] [ECOUrl ""https://www.chess.com/openings/Queens-Pawn-Opening-1...d5-2.e3""] [UTCDate ""2013.02.01""] [UTCTime ""11:24:19""] [WhiteElo ""1727""] [BlackElo ""842""] [TimeControl ""1/172800""] [Termination ""-Amos- won by resignation""] [StartTime ""11:24:19""] [EndDate ""2013.02.02""] [EndTime ""17:58:11""] [Link ""https://www.chess.com/game/daily/64714474""] 1. d4 d5 2. e3 c6 3. c4 dxc4 4. Bxc4 b5 5. Bb3 a5 6. Qf3 Bb7 7. Bxf7+ Kd7 8. Qf5+ Kc7 9. Nf3 Nh6 10. Qe5+ Kb6 11. a4 bxa4 12. Nc3 Ng4 13. Nxa4+ Ka6 14. Bc4+ Ka7 15. Qc5+ 1-0"
enhmandah,-Amos-,https://api.chess.com/pub/player/enhmandah,https://api.chess.com/pub/player/-amos-,819,1727,checkmated,win,daily,1/172800,chess,true,r3kb1r/pp3ppp/3p1n2/2pKp3/P3P3/1P6/4qP1P/QNB5 w kq - 3 17,"[Event ""CHESS BOARD CLASH - Round 1""] [Site ""Chess.com""] [Date ""2013.02.01""] [Round ""-""] [White ""enhmandah

In [0]:
bronze_table = "workspace.default.bronze_games"

(
  df_raw.write
  .format("delta")
  .mode("overwrite")
  .saveAsTable(bronze_table)
)

print("Bronze salvo em:", bronze_table)


Bronze salvo em: workspace.default.bronze_games


In [0]:
%sql
SELECT COUNT(*) AS n_rows FROM workspace.default.bronze_games;

n_rows
66879


In [0]:
%sql
SELECT * FROM workspace.default.bronze_games LIMIT 10;

white_username,black_username,white_id,black_id,white_rating,black_rating,white_result,black_result,time_class,time_control,rules,rated,fen,pgn
-Amos-,miniman2804,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/miniman2804,1708,1608,win,checkmated,daily,1/259200,chess,true,r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R b K f3 1 22,"[Event ""Enjoyable games 2 - Round 1""] [Site ""Chess.com""] [Date ""2013.01.30""] [Round ""-""] [White ""-Amos-""] [Black ""miniman2804""] [Result ""1-0""] [Tournament ""https://www.chess.com/tournament/enjoyable-games-2""] [CurrentPosition ""r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R b K f3 1 22""] [Timezone ""UTC""] [ECO ""E22""] [ECOUrl ""https://www.chess.com/openings/Nimzo-Indian-Defense-Spielmann-Variation""] [UTCDate ""2013.01.30""] [UTCTime ""16:35:14""] [WhiteElo ""1708""] [BlackElo ""1608""] [TimeControl ""1/259200""] [Termination ""-Amos- won by checkmate""] [StartTime ""16:35:14""] [EndDate ""2013.02.01""] [EndTime ""18:14:48""] [Link ""https://www.chess.com/game/daily/64629816""] 1. d4 Nf6 2. c4 e6 3. Nc3 Bb4 4. Qb3 Bxc3+ 5. Qxc3 O-O 6. Bg5 c5 7. dxc5 Nc6 8. Nf3 Qa5 9. Bxf6 gxf6 10. Qxa5 Nxa5 11. e3 Rd8 12. Rd1 Kg7 13. Be2 b6 14. Rd4 bxc5 15. Rg4+ Kh6 16. Bd3 f5 17. Rh4+ Kg6 18. g4 Ba6 19. gxf5+ exf5 20. Ne5+ Kf6 21. Rh6+ Kxe5 22. f4# 1-0"
-Amos-,koltcho69,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/koltcho69,1726,1577,win,resigned,daily,1/172800,chess,true,8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33,"[Event ""Rapid Rats - Board 5""] [Site ""Chess.com""] [Date ""2013.01.19""] [Round ""-""] [White ""-Amos-""] [Black ""koltcho69""] [Result ""1-0""] [Match ""https://www.chess.com/club/matches/219602""] [CurrentPosition ""8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33""] [Timezone ""UTC""] [ECO ""C53""] [ECOUrl ""https://www.chess.com/openings/Giuoco-Piano-Game-Main-Line""] [UTCDate ""2013.01.19""] [UTCTime ""14:29:25""] [WhiteElo ""1726""] [BlackElo ""1577""] [TimeControl ""1/172800""] [Termination ""-Amos- won by resignation""] [StartTime ""14:29:25""] [EndDate ""2013.02.01""] [EndTime ""18:22:03""] [Link ""https://www.chess.com/game/daily/64070770""] 1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5 4. c3 a6 5. d4 exd4 6. cxd4 Be7 7. Qb3 Na5 8. Qc2 Nxc4 9. Qxc4 d6 10. Nc3 c6 11. O-O h6 12. Re1 Nf6 13. d5 c5 14. e5 dxe5 15. Nxe5 O-O 16. Ng6 Re8 17. Rxe7 Rxe7 18. Nxe7+ Qxe7 19. Bf4 b5 20. d6 Qd7 21. Qxc5 Bb7 22. Qc7 Qxc7 23. dxc7 Nd5 24. Rd1 Nxf4 25. Rd8+ Kh7 26. Rxa8 Bxa8 27. c8=Q Bxg2 28. Qxa6 Bf3 29. Qxb5 Nh3+ 30. Kf1 g6 31. Qd7 Ng5 32. h4 Ne6 33. Qxf7+ 1-0"
-Amos-,enhmandah,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/enhmandah,1727,842,win,resigned,daily,1/172800,chess,true,rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/R1B1K2R b KQ - 5 15,"[Event ""CHESS BOARD CLASH - Round 1""] [Site ""Chess.com""] [Date ""2013.02.01""] [Round ""-""] [White ""-Amos-""] [Black ""enhmandah""] [Result ""1-0""] [Tournament ""https://www.chess.com/tournament/just-another-clash""] [CurrentPosition ""rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/R1B1K2R b KQ - 5 15""] [Timezone ""UTC""] [ECO ""D00""] [ECOUrl ""https://www.chess.com/openings/Queens-Pawn-Opening-1...d5-2.e3""] [UTCDate ""2013.02.01""] [UTCTime ""11:24:19""] [WhiteElo ""1727""] [BlackElo ""842""] [TimeControl ""1/172800""] [Termination ""-Amos- won by resignation""] [StartTime ""11:24:19""] [EndDate ""2013.02.02""] [EndTime ""17:58:11""] [Link ""https://www.chess.com/game/daily/64714474""] 1. d4 d5 2. e3 c6 3. c4 dxc4 4. Bxc4 b5 5. Bb3 a5 6. Qf3 Bb7 7. Bxf7+ Kd7 8. Qf5+ Kc7 9. Nf3 Nh6 10. Qe5+ Kb6 11. a4 bxa4 12. Nc3 Ng4 13. Nxa4+ Ka6 14. Bc4+ Ka7 15. Qc5+ 1-0"
enhmandah,-Amos-,https://api.chess.com/pub/player/enhmandah,https://api.chess.com/pub/player/-amos-,819,1727,checkmated,win,daily,1/172800,chess,true,r3kb1r/pp3ppp/3p1n2/2pKp3/P3P3/1P6/4qP1P/QNB5 w kq - 3 17,"[Event ""CHESS BOARD CLASH - Round 1""] [Site ""Chess.com""] [Date ""2013.02.01""] [Round ""-""] [White ""enhmandah